In [144]:
import pandas as pd
import numpy as np

In [71]:
data = pd.read_csv('insurance.csv')
data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [51]:
# Трансформируем с помощью one hot encoder
train_data = data[['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']]

one_hot = pd.get_dummies(train_data['sex'], columns='sex', prefix='sex', dtype = int)
train_data[['sex_female', 'sex_male']] = one_hot
train_data = train_data.drop(columns = 'sex')

one_hot = pd.get_dummies(train_data['smoker'], columns='smoker', prefix='smoker', dtype = int)
train_data[['not_smoker', 'is_smoker']] = one_hot
train_data = train_data.drop(columns = 'smoker')

one_hot = pd.get_dummies(train_data['region'], columns='region', prefix='region', dtype = int)
train_data[['reg_sw', 'reg_nw', 'reg_se', 'reg_ne']] = one_hot
train_data = train_data.drop(columns = 'region')

train_data

,age,bmi,children,charges,sex_female,sex_male,not_smoker,is_smoker,reg_sw,reg_nw,reg_se,reg_ne
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [117]:
# Делаем копию исходных данных. Затем ещё делим на данные с параметрами и данные с ответами.
# Переделываем данные в матрицу
data_x = train_data.drop(columns='charges')
data_y = train_data.charges
# data_x = pd.DataFrame(data_x)
# data_y = pd.DataFrame(data_y)
data_y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

In [149]:
"""pd.concat adds the two DataFrames together by appending one right after the other. if there is any overlap, it will be captured by the drop_duplicates method. However, drop_duplicates by default leaves the first observation and removes every other observation. In this case, we want every duplicate removed. Hence, the keep=False parameter which does exactly that.
A special note to the repeated df2. With only one df2 any row in df2 not in df1 won't be considered a duplicate and will remain. This solution with only one df2 only works when df2 is a subset of df1. However, if we concat df2 twice, it is guaranteed to be a duplicate and will subsequently be removed."""
train_x = data_x.sample(frac=0.8, random_state=200) #random_state - это сид рандомайзера
test_x = pd.concat([data_x, train_x, train_x]).drop_duplicates(keep=False)

train_y = data_y.sample(frac=0.8, random_state=200)
test_y = pd.concat([data_y, train_y, train_y]).drop_duplicates(keep=False)

#Разделили на тренируемые и проверяемые
test_x.head(20)



,age,bmi,children,sex_female,sex_male,not_smoker,is_smoker,reg_sw,reg_nw,reg_se,reg_ne
0,19,27.900,0,1,0,0,1,0,0,0,1
10,25,26.220,0,0,1,1,0,1,0,0,0
15,19,24.600,1,0,1,1,0,0,0,0,1
17,23,23.845,0,0,1,1,0,1,0,0,0
18,56,40.300,0,0,1,1,0,0,0,0,1
27,55,32.775,2,1,0,1,0,0,1,0,0
29,31,36.300,2,0,1,0,1,0,0,0,1
52,48,28.000,1,0,1,0,1,0,0,0,1
55,58,36.955,2,0,1,0,1,0,1,0,0
67,40,26.315,1,0,1,1,0,0,1,0,0


In [143]:
a = np.random.randn(1)
b = np.random.randn(1)
print(a,b)

[-0.78766955] [-1.10747683]


In [ ]:
def MSE(test_x, pred_y)
    MSE = np.mean((test_x - pred_y)**2)